In [1]:
import requests
import json
import time
import pandas as pd

#API Key is in the copy

class GoogleGeocode(object):
    def __init__(self, GeoapiKey):
        super(GoogleGeocode, self).__init__()
        self.apiKey = GeoapiKey
        
    def get_coordinates(self, address):
        endpoint_url = 'https://maps.googleapis.com/maps/api/geocode/json'
        params = {
            'address': address,
            'key': self.apiKey
        }
        res = requests.get(endpoint_url, params = params)
        georesults =  json.loads(res.content)
        lat = georesults['results'][0]['geometry']['location']['lat']
        lng = georesults['results'][0]['geometry']['location']['lng']
        cordinates = str(lat) + ',' + str(lng)
        return(cordinates)
        

In [2]:
class GooglePlaces(object):
    def __init__(self, apiKey):
        super(GooglePlaces, self).__init__()
        self.apiKey = apiKey
 
    def search_places_by_coordinate(self, location, types):
        endpoint_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
        places = []
        params = {
            'location': location,
            'types': types,
            'rankby' : 'distance',
            'key': self.apiKey
        }
        res = requests.get(endpoint_url, params = params)
        results =  json.loads(res.content)
        places.extend(results['results'])
        time.sleep(2)
        
        while "next_page_token" in results:
            params['pagetoken'] = results['next_page_token'],
            res = requests.get(endpoint_url, params = params)
            results = json.loads(res.content)
            places.extend(results['results'])
            time.sleep(2)
        
        return(places)

    
    def get_place_details(self, place_id, fields):
        endpoint_url = "https://maps.googleapis.com/maps/api/place/details/json"
        params = {
            'placeid': place_id,
            'fields': ",".join(fields),
            'key': self.apiKey
        }
        res = requests.get(endpoint_url, params = params)
        place_details =  json.loads(res.content)
        return(place_details)

In [3]:
geoapi = GoogleGeocode(GeoapiKey)

In [4]:
cordinates = geoapi.get_coordinates('Missoula')

In [5]:
cordinates

'46.8721284,-113.9940314'

In [6]:
api = GooglePlaces(apiKey)

In [7]:
places = api.search_places_by_coordinate(cordinates, types = 'restaurant')

In [ ]:
places

Big limitation can only resturn 60 results. They used to have a radar search that would return 200 results. It was taken down. Google has it posted all over the web that workarounds are agianst their service terms to prevent web scraping. I decided not to take this risk.

In [8]:
len(places)

60

In [9]:
fields = ['name', 'user_ratings_total', 'formatted_address', 'opening_hours', 'permanently_closed', 'price_level', 'international_phone_number', 'website', 'rating', 'review']

In [10]:
for place in places:
    try:
        details = api.get_place_details(place['place_id'], fields)
        #name = details['result']['name']
        rest_info = pd.json_normalize(details)
        restaurants = restaurants.append(rest_info)
    except:
        details = api.get_place_details(place['place_id'], fields)
        restaurants = pd.json_normalize(details)
    

In [11]:
len(restaurants)

60

In [12]:
restaurants.drop(['html_attributions',
                  'status',
                  'result.opening_hours.open_now',
                  'result.opening_hours.periods']
                 , axis=1, inplace=True)

In [13]:
restaurants.rename(columns = {'result.formatted_address':'Address', 
                              'result.international_phone_number':'Phone_Number', 
                              'result.name':'Name',
                              'result.opening_hours.weekday_text': 'Hours_Open',
                              'result.price_level' : 'Price_level',
                              'result.reviews' : 'reviews',
                              'result.website' : 'website',
                              'result.rating': 'Rating',
                              'result.user_ratings_total': '#_of_Ratings'}, inplace = True)

In [14]:
restaurants = restaurants.set_index('Name')

In [15]:
restaurants

,Address,Phone_Number,Hours_Open,Rating,reviews,#_of_Ratings,website,Price_level,result.permanently_closed
Name,,,,,,,,,
Clove Cart Pizza,"Mobile Pizza Oven, Missoula, MT 59802, USA",+1 617-571-6894,"[Monday: Closed, Tuesday: Closed, Wednesday: 1...",5.0,"[{'author_name': 'Jennifer Trimble', 'author_u...",5.0,http://www.clovecart.com/,NaN,NaN
Pangea Bar & Restaurant Missoula,"223 N Higgins Ave, Missoula, MT 59802, USA",+1 406-493-1190,"[Monday: 11:00 AM – 10:00 PM, Tuesday: 11:00 A...",4.2,"[{'author_name': 'Brian Logsdon', 'author_url'...",161.0,http://mtpangea.com/,NaN,NaN
Plonk Missoula,"322 N Higgins Ave, Missoula, MT 59802, USA",+1 406-926-1791,"[Monday: 3:00 PM – 12:00 AM, Tuesday: 3:00 PM ...",4.5,"[{'author_name': 'Gloria heintz', 'author_url'...",513.0,http://plonkwine.com/,3.0,NaN
Liquid Planet - Downtown,"121 W Broadway St, Missoula, MT 59802, USA",+1 406-541-4541,"[Monday: 7:30 AM – 8:00 PM, Tuesday: 7:30 AM –...",4.4,"[{'author_name': 'Lacy Z', 'author_url': 'http...",449.0,https://liquidplanet.com/pages/locations,2.0,NaN
Five on Black,"325 N Higgins Ave, Missoula, MT 59802, USA",+1 406-926-1860,"[Monday: 11:00 AM – 9:00 PM, Tuesday: 11:00 AM...",4.7,"[{'author_name': 'J T', 'author_url': 'https:/...",917.0,http://fiveonblack.com/,1.0,NaN
Doc's Sandwich Shop,"214 N Higgins Ave, Missoula, MT 59802, USA",+1 406-542-7414,"[Monday: 7:00 AM – 4:00 PM, Tuesday: 7:00 AM –...",4.2,"[{'author_name': 'Jesse Franzen', 'author_url'...",234.0,http://docssandwiches.com/,2.0,NaN
The Oxford Cafe,"337 N Higgins Ave, Missoula, MT 59802, USA",+1 406-549-0117,"[Monday: Open 24 hours, Tuesday: Open 24 hours...",4.3,"[{'author_name': 'Trevor Anderson', 'author_ur...",836.0,http://www.the-oxford.com/,1.0,NaN
Locals,"149 W Broadway St, Missoula, MT 59802, USA",+1 406-543-0646,"[Monday: 5:00 PM – 2:00 AM, Tuesday: 5:00 PM –...",4.0,"[{'author_name': 'Lyle Masterson', 'author_url...",22.0,https://www.facebook.com/localsonlymissoula/?r...,NaN,NaN
Sushi Hana Downtown,"403 N Higgins Ave, Missoula, MT 59802, USA",+1 406-549-7979,"[Monday: 11:30 AM – 3:00 PM, 5:00 – 9:00 PM, T...",4.2,"[{'author_name': 'Timothy Michael Ufferman', '...",621.0,http://sushimissoula.com/,2.0,NaN


In [16]:
closed = restaurants[restaurants['result.permanently_closed'] == True].index
restaurants.drop(closed, inplace = True)

In [17]:
restaurants.drop('result.permanently_closed', axis = 1, inplace= True)

In [18]:
restaurants = restaurants.sort_values('Rating',  ascending=False)

In [19]:
restaurants.to_csv('Google_Missoula_Restaurants.csv')

In [20]:
review_dictionary = {}
for key, value in restaurants.iterrows():
    rest_review = value['reviews']
    if type(rest_review) == float:
        continue
    else:
        review_df = pd.json_normalize(rest_review)
        review_dictionary[key] = review_df

In [21]:
for key in review_dictionary:
    review_dictionary[key].to_csv('Google_Reviews/' + str(key) + '_Google_reviews.csv')

In [22]:
len(restaurants)

57

In [ ]:
restaurants